In [3]:
from cnn_image_generation import *
from analysis_nn import *
from update_library import *
import random

from PIL import Image, ImageFilter
from IPython import display

from matplotlib.pyplot import imshow
%matplotlib inline

import pickle

lib = update_library()

Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with

In [4]:
import squeezedet as nn
net = nn.init()

INFO:tensorflow:Summary name mean iou is illegal; using mean_iou instead.
INFO:tensorflow:Restoring parameters from ./squeezeDet/data/model_checkpoints/squeezeDet/model.ckpt-87000


In [5]:
num_pics = 975
# Sampling bounds (x,y)
sample_dim = 2
images_domain = [[52,72],[1,19]]
modification_domain = [[0,1],[0.35,1]]

#path_synth_set = './synth/uniform/'
path_synth_set = './synth/uniform_misclass/'

err_table = error_table()

In [6]:
def scale_sample(sample, sampling_domain):
    '''Scale a [0,1] sample in a given domain'''
    for i in range(len(sample)):
        sample[i] = sample[i]*(sampling_domain[i][1] - sampling_domain[i][0]) + sampling_domain[i][0]
    return sample    

def random_sample(typ,sampling_domain):
    sample = []
    for i in range(len(sampling_domain)):
        domain = sampling_domain[i]
        if typ == 'float':
            r = random.random()*(domain[1]-domain[0])+domain[0]
        elif typ == 'int':
            r = random.choice(range(domain[0],domain[1]))
        else:
            print('Error')
        sample.append(r)
    return sample

def box_2_kitti_format(box):
    '''Transform box for KITTI label format'''
    x = box[0]
    y = box[1]
    w = box[2]
    h = box[3]
    left = int(x - w/2)
    right = int(x + w/2)
    top = int(y - h/2)
    bot = int(y + h/2)
    return [left,top,right,bot]
    

In [8]:
#for i in range(num_pics):
i = 0
while i < num_pics:
    if (i % 25 == 0):
        print(i)
    
    sample = random_sample('int',images_domain) + random_sample('float',modification_domain)
    img, box, img_info = gen_comp_img(lib, x=sample[2], y=sample[3], bg_id=sample[0], fg_id=sample[1])
    
    # Save tmp image    
    
    tmp_img_file_name = './tmp.png'
    img.save(tmp_img_file_name)
    
    # Classify img
    (boxes,probs,cats) = nn.classify(tmp_img_file_name,net) 
    
    save_img = False
    # More than one category
    if len(cats) != 1:
        save_img = True
    # Not a car
    elif cats[0] != 0:
        save_img = True
    elif (nn.iou(box, boxes[0])*probs[0]) < 0.5:
        save_img = True
        
    if save_img:
        file_name = 's' + str(i).zfill(6)
        
        # Save image
        img_file_name = path_synth_set + 'images/' + file_name + '.png'
        img.save(img_file_name)
    
        # Save label
        f = open(path_synth_set + 'labels/' + file_name + '.txt', 'w')
        label = [0,0,0] + box_2_kitti_format(box) + [0,0,0,0,0,0,0]
        label = list(map(str, label))
        label = " ".join(label)
        label  = "Car " + label
        f.write(label)  # python will convert \n to os.linesep
        f.close()
        
        err_table.update_with_elem(img_info)
        
        i+=1

0
25
50
75
100
125
150
175
200
225
250
275
300
325
350
375
400
425
450
475
500
525
550
575
600
625
650
675
700
725
750
775
800
825
850
875
900
925
950
